In [1]:
import os
os.chdir('..')
from MPRA_predict.utils import *

In [2]:
cell_types = ['HepG2', 'K562', 'SK-N-SH', 'A549', 'HCT116']
assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']

In [3]:
MPRA_df = pd.read_csv('data/SirajMPRA/SirajMPRA_562654_stack.csv')
MPRA_df

,description,seq,A549,HepG2,K562,SK-N-SH,HCT116,is_ref,chr,pos,ref,alt,start,end,585605_index,HepG2_DNase,HepG2_H3K4me1,HepG2_H3K4me3,HepG2_H3K9me3,HepG2_H3K27me3,HepG2_H3K27ac,HepG2_H3K36me3,HepG2_CTCF,K562_DNase,K562_H3K4me1,K562_H3K4me3,K562_H3K9me3,K562_H3K27me3,K562_H3K27ac,K562_H3K36me3,K562_CTCF,SK-N-SH_DNase,SK-N-SH_H3K4me1,SK-N-SH_H3K4me3,SK-N-SH_H3K9me3,SK-N-SH_H3K27me3,SK-N-SH_H3K27ac,SK-N-SH_H3K36me3,SK-N-SH_CTCF,A549_DNase,A549_H3K4me1,A549_H3K4me3,A549_H3K9me3,A549_H3K27me3,A549_H3K27ac,A549_H3K36me3,A549_CTCF,HCT116_DNase,HCT116_H3K4me1,HCT116_H3K4me3,HCT116_H3K9me3,HCT116_H3K27me3,HCT116_H3K27ac,HCT116_H3K36me3,HCT116_CTCF
0,chr1:14677:G:A,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.256,-1.092,0.156,-0.223,True,chr1,14677,G,A,14577,14777,0,1.313,1.209,1.923,0.779,2.484,1.661,2.092,1.207,1.434,1.857,2.131,2.178,2.211,1.846,1.891,1.453,1.430,NaN,2.076,NaN,2.219,1.373,1.631,1.184,1.121,1.384,2.139,2.259,2.374,1.727,1.780,1.287,0.921,1.231,2.038,0.887,1.550,1.624,1.946,1.171
1,chr1:14677:G:A,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,-0.003,0.068,0.066,-0.719,False,chr1,14677,G,A,14577,14777,1,1.294,1.236,1.913,0.762,2.495,1.664,2.110,1.235,1.412,1.844,2.114,2.147,2.189,1.854,1.864,1.473,1.373,NaN,2.010,NaN,2.215,1.342,1.631,1.187,1.108,1.362,2.092,2.203,2.313,1.698,1.719,1.309,0.913,1.179,2.004,0.789,1.485,1.597,1.908,1.197
2,chr1:63697:T:C,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,0.877,-0.803,0.354,0.821,True,chr1,63697,T,C,63597,63797,2,1.026,1.371,1.174,0.130,0.524,0.922,0.086,0.409,0.319,0.198,0.462,-0.438,0.668,0.157,0.617,0.078,-0.110,NaN,1.122,NaN,-0.050,1.787,0.537,0.378,1.392,1.241,1.204,0.145,0.882,0.838,0.937,0.314,0.911,1.095,0.640,-0.177,1.205,0.795,0.211,0.206
3,chr1:63697:T:C,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,0.756,-1.050,-0.154,0.766,False,chr1,63697,T,C,63597,63797,3,1.022,1.274,0.990,0.335,0.480,0.783,0.053,0.411,0.224,0.079,0.305,-0.415,0.611,0.004,0.595,0.037,-0.260,NaN,0.820,NaN,-0.022,1.525,0.580,0.285,1.384,1.097,0.969,0.173,0.751,0.632,0.989,0.298,0.925,1.043,0.513,-0.162,0.894,0.676,0.277,0.188
4,chr1:64764:C:T,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.649,-0.449,0.119,-0.114,True,chr1,64764,C,T,64664,64864,4,-0.988,-0.600,-0.719,0.185,-0.474,-0.858,-0.848,-0.901,-0.765,-0.785,-0.781,-1.259,-0.531,-0.662,-0.968,-0.709,-0.835,NaN,-0.579,NaN,-0.892,-0.424,-1.130,-0.776,-1.083,-0.762,-0.813,-0.991,0.105,-0.803,-0.624,-0.821,-0.935,-0.916,-0.818,0.435,0.415,-0.736,-1.276,-0.686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562649,chrX:155233098:T:C,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,NaN,0.203,1.334,0.397,0.749,False,chrX,155233098,T,C,155232998,155233198,585600,2.066,2.489,2.190,0.419,1.252,2.592,1.194,1.500,1.432,1.627,1.429,1.534,1.026,1.501,1.212,1.084,1.033,NaN,0.972,NaN,1.426,0.688,1.553,0.994,1.518,2.130,2.014,0.877,0.901,2.117,0.660,1.190,1.159,1.561,1.521,-1.286,0.266,1.322,1.346,0.979
562650,chrX:155233794:A:G,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,1.021,0.070,2.294,1.259,True,chrX,155233794,A,G,155233694,155233894,585601,0.844,0.886,1.232,0.873,0.177,0.820,1.057,0.878,0.668,0.424,0.696,0.436,0.450,0.611,1.729,0.373,1.408,NaN,1.844,NaN,0.849,2.360,1.388,1.255,1.744,1.724,1.473,0.836,0.536,1.525,1.424,0.911,1.858,1.976,1.727,-0.630,1.291,1.653,1.009,0.885
562651,chrX:155233794:A:G,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,0.490,-0.510,1.487,0.482,False,chrX,155233794,A,G,155233694,155233894,585602,0.810,1.034,1.391,0.815,0.714,0.957,1.205,0.843,0.601,0.528,0.870,0.488,0.796,0.746,1.662,0.405,1.332,NaN,1.759,NaN,1.025,1.878,1.365,1.126,1.374,1.373,1.496,0.829,1.060,1.356,1.477,0.833,1.471,1.646,1.782,-0.341,1.285,1.487,1.148,0.733
562652,chrX:155234730:T:C,TGGTGAGTTGCTTCTGTGCACACACTTTTGTTTATGAGTGTGCAT

In [4]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(MPRA_df), dtype=bool)
data_split_dict['chrom'] = MPRA_df['chr'].isin([f'chr{i}' for i in range(1, 23)])
data_split_dict['train'] = MPRA_df['chr'].isin([f'chr{i}' for i in range(3, 23)])
data_split_dict['valid'] = MPRA_df['chr'].isin(['chr1'])
data_split_dict['test'] = MPRA_df['chr'].isin(['chr2'])
data_split_dict['designed'] = MPRA_df['chr'].isin(['chr0'])


stds = MPRA_df[cell_types].std(axis=1, skipna=True)
threshold = np.percentile(stds.dropna(), 90)
data_split_dict['diff'] = (stds > threshold)
data_split_dict['const'] = ~data_split_dict['diff']

# MPRA_df['diff'] = (MPRA_df[cell_types].max(axis=1) - MPRA_df[cell_types].min(axis=1)) > 1
# data_split_dict['const'] = ~MPRA_df['diff']
# data_split_dict['diff']  = MPRA_df['diff']
print(len(MPRA_df), data_split_dict['const'].sum(), data_split_dict['diff'].sum())


keys = list(data_split_dict.keys())
for k1 in keys:
    for k2 in keys:
        data_split_dict[f'{k1}_{k2}'] = data_split_dict[k1] & data_split_dict[k2]

562654 506509 56145


In [5]:
# 各细胞类型的真实活性的相关性

for data_split in ['train', 'valid', 'test', 'const', 'diff', 'test_const', 'test_diff']:
    r_array = pd.DataFrame(index=cell_types, columns=cell_types)
    for cell_type in cell_types:
        for cell_type_2 in cell_types:
            condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            x, y = remove_nan(df[f'{cell_type}'], df[f'{cell_type_2}'])
            r, p = pearsonr(x, y)
            r_array.loc[cell_type, cell_type_2] = r
    print(data_split)
    print(r_array)

train
        HepG2  K562 SK-N-SH  A549 HCT116
HepG2   1.000 0.788   0.827 0.740  0.868
K562    0.788 1.000   0.739 0.728  0.808
SK-N-SH 0.827 0.739   1.000 0.778  0.816
A549    0.740 0.728   0.778 1.000  0.817
HCT116  0.868 0.808   0.816 0.817  1.000
valid
        HepG2  K562 SK-N-SH  A549 HCT116
HepG2   1.000 0.778   0.821 0.739  0.859
K562    0.778 1.000   0.735 0.735  0.805
SK-N-SH 0.821 0.735   1.000 0.784  0.811
A549    0.739 0.735   0.784 1.000  0.804
HCT116  0.859 0.805   0.811 0.804  1.000
test
        HepG2  K562 SK-N-SH  A549 HCT116
HepG2   1.000 0.770   0.806 0.727  0.858
K562    0.770 1.000   0.726 0.708  0.802
SK-N-SH 0.806 0.726   1.000 0.769  0.802
A549    0.727 0.708   0.769 1.000  0.813
HCT116  0.858 0.802   0.802 0.813  1.000
const
        HepG2  K562 SK-N-SH  A549 HCT116
HepG2   1.000 0.845   0.858 0.777  0.891
K562    0.845 1.000   0.797 0.769  0.846
SK-N-SH 0.858 0.797   1.000 0.764  0.832
A549    0.777 0.769   0.764 1.000  0.796
HCT116  0.891 0.846   0.832 0.796 

In [6]:
# seq
# train 5 cell types

y_pred = np.load(f'saved/0304_SirajMPRA_seq/0304_033552/test_pred.npy')
cell_types_pred = [f'{cell_type}_pred' for cell_type in cell_types]
MPRA_df[cell_types_pred] = y_pred

In [7]:
for data_split in ['train', 'valid', 'test', 'const', 'diff', 'test_const', 'test_diff']:
    r_array = pd.DataFrame(index=cell_types, columns=cell_types_pred)
    for cell_type in cell_types:
        for cell_type_pred in cell_types_pred:
            condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            x = df[f'{cell_type}']
            y = df[f'{cell_type_pred}']
            x, y = remove_nan(x, y)
            r, p = pearsonr(x, y)
            r_array.loc[cell_type, cell_type_pred] = r
    print(data_split)
    print(r_array)

train
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2        0.927     0.811        0.850     0.809       0.864
K562         0.824     0.932        0.785     0.811       0.835
SK-N-SH      0.852     0.773        0.921     0.841       0.826
A549         0.783     0.757        0.811     0.889       0.785
HCT116       0.877     0.838        0.838     0.833       0.931
valid
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2        0.842     0.730        0.765     0.720       0.777
K562         0.754     0.849        0.714     0.748       0.760
SK-N-SH      0.775     0.695        0.840     0.754       0.739
A549         0.713     0.692        0.733     0.804       0.708
HCT116       0.795     0.758        0.751     0.756       0.850
test
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2        0.839     0.723        0.760     0.717       0.769
K562         0.747     0.844        0.711     0.737       0.752
SK-N-SH      0.760     

In [8]:
# seq + feature
# train 5 cell types

y_pred = np.load(f'saved/0305_SirajMPRA_seq_feature_matrix/0304_033430/test_pred.npy')
cell_types_pred = [f'{cell_type}_pred' for cell_type in cell_types]
MPRA_df[cell_types_pred] = y_pred

In [9]:
for data_split in ['train', 'valid', 'test', 'const', 'diff', 'test_const', 'test_diff']:
    r_array = pd.DataFrame(index=cell_types, columns=cell_types_pred)
    for cell_type in cell_types:
        for cell_type_pred in cell_types_pred:
            condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            x = df[f'{cell_type}']
            y = df[f'{cell_type_pred}']
            x, y = remove_nan(x, y)
            r, p = pearsonr(x, y)
            r_array.loc[cell_type, cell_type_pred] = r
    print(data_split)
    print(r_array)

train
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2        0.901     0.875        0.864     0.882       0.879
K562         0.838     0.871        0.827     0.843       0.843
SK-N-SH      0.868     0.868        0.889     0.878       0.875
A549         0.835     0.835        0.836     0.864       0.857
HCT116       0.890     0.879        0.874     0.894       0.900
valid
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2        0.824     0.801        0.786     0.807       0.805
K562         0.768     0.808        0.759     0.778       0.780
SK-N-SH      0.787     0.792        0.812     0.801       0.799
A549         0.738     0.745        0.741     0.771       0.767
HCT116       0.808     0.800        0.794     0.818       0.826
test
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2        0.817     0.789        0.776     0.800       0.796
K562         0.759     0.798        0.752     0.769       0.772
SK-N-SH      0.770     

In [10]:
# seq
# train 4 cell types, valid 1 cell type






In [20]:
# seq + feature
# train 4 cell types, valid 1 cell type

y_pred = np.zeros((len(MPRA_df), 5))
y_pred[:, 0] = np.load(f'saved/0306_SirajMPRA_seq_feature_without_HepG2/0306_041237/test_pred.npy')[:, 0]
y_pred[:, 1] = np.load(f'saved/0306_SirajMPRA_seq_feature_without_K562/0306_050958/test_pred.npy')[:, 1]
y_pred[:, 2] = np.load(f'saved/0306_SirajMPRA_seq_feature_without_SK-N-SH/0306_061734/test_pred.npy')[:, 2]
y_pred[:, 3] = np.load(f'saved/0306_SirajMPRA_seq_feature_without_A549/0307_010836/test_pred.npy')[:, 3]
y_pred[:, 4] = np.load(f'saved/0306_SirajMPRA_seq_feature_without_HCT116/0306_080925/test_pred.npy')[:, 4]

cell_types_pred = [f'{cell_type}_pred' for cell_type in cell_types]
MPRA_df[cell_types_pred] = y_pred

In [21]:
for data_split in ['train', 'valid', 'test', 'const', 'diff', 'test_const', 'test_diff']:
    r_array = pd.DataFrame(index=cell_types, columns=cell_types_pred)
    for cell_type in cell_types:
        for cell_type_pred in cell_types_pred:
            condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            x = df[f'{cell_type}']
            y = df[f'{cell_type_pred}']
            x, y = remove_nan(x, y)
            r, p = pearsonr(x, y)
            r_array.loc[cell_type, cell_type_pred] = r
    print(data_split)
    print(r_array)

train
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2        0.852     0.875        0.829     0.880       0.861
K562         0.844     0.834        0.826     0.850       0.843
SK-N-SH      0.850     0.868        0.809     0.863       0.864
A549         0.848     0.850        0.810     0.812       0.862
HCT116       0.885     0.888        0.860     0.895       0.864
valid
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2        0.785     0.799        0.745     0.801       0.784
K562         0.771     0.788        0.748     0.780       0.778
SK-N-SH      0.766     0.788        0.739     0.787       0.786
A549         0.742     0.755        0.707     0.754       0.773
HCT116       0.800     0.808        0.774     0.817       0.802
test
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2        0.779     0.790        0.730     0.802       0.782
K562         0.764     0.778        0.732     0.775       0.772
SK-N-SH      0.755     

In [23]:
y_pred_4_0 = np.load(f'saved/0812_SirajMPRA_seq_celltype4/0814_053219/test_pred.npy')
print(y_pred_4_0.shape)

y_pred_4_f = np.load(f'saved/0812_SirajMPRA_seq_feature_celltype4/0815_041254/test_pred.npy')
print(y_pred_4_f.shape)

y_pred_3_H = np.load(f'saved/0815_SirajMPRA_seq_feature_celltype3/0815_041826/test_pred.npy')
print(y_pred_3_H.shape)

y_pred_3_A = np.load(f'saved/0815_SirajMPRA_seq_feature_celltype3/0819_005952/test_pred.npy')
print(y_pred_3_A.shape)

y_pred_3_K = np.load(f'saved/0815_SirajMPRA_seq_feature_celltype3/0815_214824/test_pred.npy')
print(y_pred_3_K.shape)

y_pred_3_H = np.load(f'saved/0815_SirajMPRA_seq_feature_celltype3/0815_214850/test_pred.npy')
print(y_pred_3_H.shape)

(585605, 4)
(1765218,)
(1765218,)
(1765218,)
(1765218,)
(1765218,)


In [25]:
MPRA_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_total.csv')
# print(MPRA_df.head())

df2 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/MPRA_exp/pretrained_based_models/data/Sei_Siraj_features_concat.csv')
# print(df2.head())


# 下面二选一

for i, cell_type in enumerate(['HepG2', 'K562', 'A549', 'HCT116']):
    MPRA_df[f'{cell_type}_pred'] = y_pred_4_0[:, i]
MPRA_df


# df2['pred'] = y_pred_4_0
# for i, cell_type in enumerate(['HepG2', 'K562', 'A549', 'HCT116']):
#     df = df2[df2['cell_type'] == cell_type][['seq', 'pred']]
#     df.rename(columns={'pred': f'{cell_type}_pred'}, inplace=True)
#     df = df.drop_duplicates(subset=['seq'])
#     MPRA_df = pd.merge(MPRA_df, df, on='seq', how='left')
# MPRA_df

,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,HepG2_pred,K562_pred,A549_pred,HCT116_pred
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677,G,A,0.860727,0.471197,0.197887,0.369649
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677,G,A,0.745834,0.418501,0.144777,0.290741
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697,T,C,0.044020,-0.045745,-0.043967,-0.115304
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697,T,C,0.009691,-0.069309,-0.071219,-0.152117
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764,C,T,-0.129289,-0.136357,-0.108981,-0.104691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585600,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,NaN,0.615976,1.927090,0.798263,1.035440,False,chrX,155233098,T,C,-0.130473,0.371278,-0.105545,-0.168190
585601,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,1.409758,0.556338,2.843295,1.495515,True,chrX,155233794,A,G,1.410156,0.740903,0.845696,0.935793
585602,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,0.894634,-0.072647,1.973661,0.794848,False,chrX,155233794,A,G,1.131462,0.488258,0.668862,0.834238
585603,TGGTGAGTTGCTTCTGTGCACACACTTTTGTTTATGAGTGTGCATG...,NaN,0.312172,0.165634,-0.041383,0.444472,True,chrX,155234730,T,C,0.393603,0.419445,0.133474,0.314708


In [26]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(MPRA_df), dtype=bool)
data_split_dict['chrom'] = MPRA_df['chr'].isin([f'chr{i}' for i in range(1, 23)])
data_split_dict['train'] = MPRA_df['chr'].isin([f'chr{i}' for i in range(3, 23)])
data_split_dict['valid'] = MPRA_df['chr'].isin(['chr1'])
data_split_dict['test'] = MPRA_df['chr'].isin(['chr2'])
data_split_dict['designed'] = MPRA_df['chr'].isin(['chr0'])


cell_types = ['HepG2', 'K562', 'A549', 'HCT116']
MPRA_df['diff'] = (MPRA_df[cell_types].max(axis=1) - MPRA_df[cell_types].min(axis=1)) > 1
data_split_dict['const'] = ~MPRA_df['diff']
data_split_dict['diff']  = MPRA_df['diff']
print(len(MPRA_df), data_split_dict['const'].sum(), data_split_dict['diff'].sum())


# MPRA_df['mean'] = MPRA_df[cell_types].mean(axis=1)
# MPRA_df['std'] = MPRA_df[cell_types].std(axis=1)

# threshold = MPRA_df['std'].quantile(0.2)
# data_split_dict['diff'] = (MPRA_df['std'] >= threshold)
# data_split_dict['const'] = (MPRA_df['std'] < threshold)

# print(data_split_dict['diff'].sum(), data_split_dict['const'].sum())

# # for cell_type in cell_types:
# #     # MPRA_df[f'{cell_type}_zscore'] = (MPRA_df[cell_type] - MPRA_df['mean']) / MPRA_df['std']
# #     # MPRA_df[f'{cell_type}_diff'] = MPRA_df[f'{cell_type}_zscore'].abs() > 1
# #     MPRA_df[f'{cell_type}_diff'] = (MPRA_df[cell_type] - MPRA_df['mean']).abs() > 1
# #     MPRA_df[f'{cell_type}_const'] = ~MPRA_df[f'{cell_type}_diff']
# #     print(f"{cell_type}_diff num = {MPRA_df[f'{cell_type}_diff'].sum()}")

In [28]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in cell_types:
        for data_split in ['total', 'train', 'valid', 'test', 'const', 'diff']:
            condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}'])
            print(f'{cell_type:6} true vs {cell_type_pred:6} true, {data_split:5}, pearson r = {r:.3f}')
        print()

HepG2  true vs HepG2  true, total, pearson r = 1.000
HepG2  true vs HepG2  true, train, pearson r = 1.000


HepG2  true vs HepG2  true, valid, pearson r = 1.000
HepG2  true vs HepG2  true, test , pearson r = 1.000
HepG2  true vs HepG2  true, const, pearson r = 1.000
HepG2  true vs HepG2  true, diff , pearson r = 1.000

HepG2  true vs K562   true, total, pearson r = 0.787
HepG2  true vs K562   true, train, pearson r = 0.789
HepG2  true vs K562   true, valid, pearson r = 0.779
HepG2  true vs K562   true, test , pearson r = 0.770
HepG2  true vs K562   true, const, pearson r = 0.833
HepG2  true vs K562   true, diff , pearson r = 0.651

HepG2  true vs A549   true, total, pearson r = 0.738
HepG2  true vs A549   true, train, pearson r = 0.740
HepG2  true vs A549   true, valid, pearson r = 0.737
HepG2  true vs A549   true, test , pearson r = 0.724
HepG2  true vs A549   true, const, pearson r = 0.705
HepG2  true vs A549   true, diff , pearson r = 0.656

HepG2  true vs HCT116 true, total, pearson r = 0.867
HepG2  true vs HCT116 true, train, pearson r = 0.869
HepG2  true vs HCT116 true, valid, pearson 

In [29]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in [cell_type]:
        for data_split in ['total', 'train', 'valid', 'test', 'const', 'diff']:
            condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}_pred'])
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split}, pearson r = {r:.3f}')
        print()

HepG2 true vs HepG2 pred, total, pearson r = 0.895
HepG2 true vs HepG2 pred, train, pearson r = 0.911
HepG2 true vs HepG2 pred, valid, pearson r = 0.813
HepG2 true vs HepG2 pred, test, pearson r = 0.811
HepG2 true vs HepG2 pred, const, pearson r = 0.858
HepG2 true vs HepG2 pred, diff, pearson r = 0.881

K562 true vs K562 pred, total, pearson r = 0.905
K562 true vs K562 pred, train, pearson r = 0.922
K562 true vs K562 pred, valid, pearson r = 0.823
K562 true vs K562 pred, test, pearson r = 0.816
K562 true vs K562 pred, const, pearson r = 0.849
K562 true vs K562 pred, diff, pearson r = 0.898

A549 true vs A549 pred, total, pearson r = 0.847
A549 true vs A549 pred, train, pearson r = 0.864
A549 true vs A549 pred, valid, pearson r = 0.767
A549 true vs A549 pred, test, pearson r = 0.760
A549 true vs A549 pred, const, pearson r = 0.728
A549 true vs A549 pred, diff, pearson r = 0.856

HCT116 true vs HCT116 pred, total, pearson r = 0.901
HCT116 true vs HCT116 pred, train, pearson r = 0.918
HCT

In [30]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in [cell_type]:
        for data_split in ['test']:
            for data_split_2 in ['total', 'const', 'diff']:
                condition = data_split_dict[data_split] & data_split_dict[data_split_2] #MPRA_df[f'{cell_type}_{data_split_2}']
                df = MPRA_df[condition]
                r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}_pred'])
                print(f'{cell_type:6} true vs {cell_type_pred:6} pred, {data_split:5}, {data_split_2:4}, pearson r = {r:.3f}')
        print()

HepG2  true vs HepG2  pred, test , total, pearson r = 0.811
HepG2  true vs HepG2  pred, test , const, pearson r = 0.768
HepG2  true vs HepG2  pred, test , diff, pearson r = 0.783

K562   true vs K562   pred, test , total, pearson r = 0.816
K562   true vs K562   pred, test , const, pearson r = 0.749
K562   true vs K562   pred, test , diff, pearson r = 0.791

A549   true vs A549   pred, test , total, pearson r = 0.760
A549   true vs A549   pred, test , const, pearson r = 0.612
A549   true vs A549   pred, test , diff, pearson r = 0.762

HCT116 true vs HCT116 pred, test , total, pearson r = 0.822
HCT116 true vs HCT116 pred, test , const, pearson r = 0.804
HCT116 true vs HCT116 pred, test , diff, pearson r = 0.810



In [31]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in cell_types:
        for data_split in ['total', 'train', 'valid', 'test']:
            for data_split_2 in ['total', 'const', 'diff']:
                condition = data_split_dict[data_split] & data_split_dict[data_split_2]
                df = MPRA_df[condition]
                r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}_pred'])
                print(f'{cell_type} true vs {cell_type_pred} pred, {data_split}, {data_split_2}, pearson r = {r:.3f}')
            print()

HepG2 true vs HepG2 pred, total, total, pearson r = 0.895
HepG2 true vs HepG2 pred, total, const, pearson r = 0.858
HepG2 true vs HepG2 pred, total, diff, pearson r = 0.881

HepG2 true vs HepG2 pred, train, total, pearson r = 0.911
HepG2 true vs HepG2 pred, train, const, pearson r = 0.876
HepG2 true vs HepG2 pred, train, diff, pearson r = 0.902

HepG2 true vs HepG2 pred, valid, total, pearson r = 0.813
HepG2 true vs HepG2 pred, valid, const, pearson r = 0.770
HepG2 true vs HepG2 pred, valid, diff, pearson r = 0.785

HepG2 true vs HepG2 pred, test, total, pearson r = 0.811
HepG2 true vs HepG2 pred, test, const, pearson r = 0.768
HepG2 true vs HepG2 pred, test, diff, pearson r = 0.783

HepG2 true vs K562 pred, total, total, pearson r = 0.790
HepG2 true vs K562 pred, total, const, pearson r = 0.791
HepG2 true vs K562 pred, total, diff, pearson r = 0.697

HepG2 true vs K562 pred, train, total, pearson r = 0.806
HepG2 true vs K562 pred, train, const, pearson r = 0.810
HepG2 true vs K562 pre

In [18]:
# saved_dir = 'saved/0625_SirajMPRA_A549/0625_025532'

HepG2_pred = np.load(f'saved/0531_GosaiMPRA_baseline/0613_231345/y_pred_list.npy')
K562_pred = np.load(f'saved/0531_GosaiMPRA_baseline/0618_001655/y_pred_list.npy')
A549_pred = np.load(f'saved/0625_SirajMPRA_A549/0625_025532/y_pred_list.npy')
HCT116_pred = np.load(f'saved/0625_SirajMPRA_HCT116/0625_025546/y_pred_list.npy')
y_pred_list = np.concatenate([HepG2_pred, K562_pred, A549_pred, HCT116_pred], axis=1)
# print(y_pred_list.shape)

MPRA_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/Siraj_Gosai_concat.csv')
for i, cell_type in enumerate(['HepG2', 'K562', 'A549', 'HCT116']):
    MPRA_df[f'{cell_type}_pred'] = y_pred_list[:, i]
MPRA_df

/tmp/ipykernel_694925/4012534825.py:10: DtypeWarning: Columns (6,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/Siraj_Gosai_concat.csv')


,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,origin,HepG2_pred,K562_pred,A549_pred,HCT116_pred
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677.0,G,A,NaN,0.603010,0.979300,0.089610,0.228658
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677.0,G,A,NaN,0.523534,0.916531,0.029055,0.214567
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697.0,T,C,NaN,-0.025966,-0.583377,0.334277,-0.240863
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697.0,T,C,NaN,-0.049796,-0.607013,0.293812,-0.277427
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764.0,C,T,NaN,-0.114398,-0.093055,-0.052438,-0.148054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661768,CTCGGAGTTCAAGAAAATCATCTCACACGGCCGTCTCAGACCCTTC...,NaN,0.591628,-0.315267,0.068697,NaN,NaN,chr0,NaN,NaN,NaN,control,1.488066,-0.080938,0.131021,-0.371440
661769,GCAATCCTACCCAGACAGATGCAGACAGCGACCTGGTGGGGGATGT...,NaN,-0.359782,-0.134821,-0.022281,NaN,NaN,chr0,NaN,NaN,NaN,control,-0.775348,-0.264292,0.012704,-0.667063
661770,CCTATGCTGCCTCCTATGCCCTTGTCATCATTGCCATTTTTGAGCT...,NaN,0.489900,0.646673,0.462366,NaN,NaN,chr0,NaN,NaN,NaN,control,0.356786,0.398112,0.177435,0.297358
661771,AGCCACAACAAAACAGCCAGTTAATGTGGAAACAAGGTCGACAACT...,NaN,-0.014411,-0.061676,0.239693,NaN,NaN,chr0,NaN,NaN,NaN,control,1.167894,0.079996,0.012424,0.225529


In [21]:
for cell_type in ['HepG2', 'K562', 'A549', 'HCT116']:
    for cell_type_pred in ['HepG2', 'K562', 'A549', 'HCT116']:
        for data_split in ['train', 'valid', 'test', 'designed']:
            condition = data_split_condition_dict[data_split]
            r = pearson(MPRA_df[condition][f'{cell_type}'], MPRA_df[condition][f'{cell_type_pred}_pred'])
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split} data, pearson r = {r:.3f}')
            print('pearson r', r)
        print()

HepG2 true vs HepG2 pred, train data, pearson r = 0.926
pearson r 0.9259957006812582
HepG2 true vs HepG2 pred, valid data, pearson r = 0.817
pearson r 0.8171515482957136
HepG2 true vs HepG2 pred, test data, pearson r = 0.815
pearson r 0.8148071042476853
HepG2 true vs HepG2 pred, designed data, pearson r = 0.745
pearson r 0.744512519022009

HepG2 true vs K562 pred, train data, pearson r = 0.759
pearson r 0.7587345782678894
HepG2 true vs K562 pred, valid data, pearson r = 0.690
pearson r 0.6903916498834725
HepG2 true vs K562 pred, test data, pearson r = 0.685
pearson r 0.6847371073960745
HepG2 true vs K562 pred, designed data, pearson r = -0.296
pearson r -0.2956059602354766

HepG2 true vs A549 pred, train data, pearson r = 0.632
pearson r 0.6322560229429839
HepG2 true vs A549 pred, valid data, pearson r = 0.583
pearson r 0.582653973730981
HepG2 true vs A549 pred, test data, pearson r = 0.574
pearson r 0.573519726677123
HepG2 true vs A549 pred, designed data, pearson r = 0.033
pearson r 

In [52]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
        for data_split in ['train', 'valid', 'test', 'designed']:
            condition = data_split_condition_dict[data_split]
            x = MPRA_df[condition][f'{cell_type}']

            cell_types_pred = [f'{ct}' for ct in cell_types if ct!= cell_type]
            y = MPRA_df[condition][cell_types_pred].mean(axis=1)
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_types_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()

HepG2 true vs ['K562', 'A549', 'HCT116'] pred, train data, pearson r = 0.852
HepG2 true vs ['K562', 'A549', 'HCT116'] pred, valid data, pearson r = 0.843
HepG2 true vs ['K562', 'A549', 'HCT116'] pred, test data, pearson r = 0.836
HepG2 true vs ['K562', 'A549', 'HCT116'] pred, designed data, pearson r = -0.292

K562 true vs ['HepG2', 'A549', 'HCT116'] pred, train data, pearson r = 0.819
K562 true vs ['HepG2', 'A549', 'HCT116'] pred, valid data, pearson r = 0.815
K562 true vs ['HepG2', 'A549', 'HCT116'] pred, test data, pearson r = 0.804
K562 true vs ['HepG2', 'A549', 'HCT116'] pred, designed data, pearson r = -0.292

A549 true vs ['HepG2', 'K562', 'HCT116'] pred, train data, pearson r = 0.779
A549 true vs ['HepG2', 'K562', 'HCT116'] pred, valid data, pearson r = 0.782
A549 true vs ['HepG2', 'K562', 'HCT116'] pred, test data, pearson r = 0.762
0 (less than 10%) values are non-nan.
A549 true vs ['HepG2', 'K562', 'HCT116'] pred, designed data, pearson r = nan

HCT116 true vs ['HepG2', 'K56

In [24]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
        for data_split in ['train', 'valid', 'test', 'designed']:
            condition = data_split_condition_dict[data_split]
            x = MPRA_df[condition][f'{cell_type}']

            cell_types_pred = [f'{ct}_pred' for ct in cell_types if ct!= cell_type]
            y = MPRA_df[condition][cell_types_pred].mean(axis=1)
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_types_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()

HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, train data, pearson r = 0.810
HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, valid data, pearson r = 0.727
HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, test data, pearson r = 0.723
HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, designed data, pearson r = -0.133

K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, train data, pearson r = 0.811
K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, valid data, pearson r = 0.745
K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, test data, pearson r = 0.740
K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, designed data, pearson r = -0.147

A549 true vs ['HepG2_pred', 'K562_pred', 'HCT116_pred'] pred, train data, pearson r = 0.762
A549 true vs ['HepG2_pred', 'K562_pred', 'HCT116_pred'] pred, valid data, pearson r = 0.708
A549 true vs ['HepG2_pred', 'K562_pred', 'HCT116_pred'] pred, test data,

In [60]:
for cell_type in ['HepG2', 'K562', 'A549', 'HCT116']:
    for cell_type_pred in ['HepG2', 'K562', 'A549', 'HCT116']:
        for data_split in ['test']:
            condition = data_split_condition_dict[data_split] & ((MPRA_df['HepG2'] - MPRA_df['K562']).abs() > 2)
            x = MPRA_df[condition][f'{cell_type}']
            y = MPRA_df[condition][f'{cell_type_pred}_pred']
            print(len(x))
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()


for cell_type in ['HepG2', 'K562', 'A549', 'HCT116']:
    for cell_type_pred in ['HepG2', 'K562', 'A549', 'HCT116']:
        for data_split in ['test']:
            condition = data_split_condition_dict[data_split] & ((MPRA_df['HepG2'] - MPRA_df['K562']).abs() <= 2)
            x = MPRA_df[condition][f'{cell_type}']
            y = MPRA_df[condition][f'{cell_type_pred}_pred']
            print(len(x))
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()


880
HepG2 true vs HepG2 pred, test data, pearson r = 0.670

880
HepG2 true vs K562 pred, test data, pearson r = 0.044

880
HepG2 true vs A549 pred, test data, pearson r = 0.166

880
HepG2 true vs HCT116 pred, test data, pearson r = 0.258

880
K562 true vs HepG2 pred, test data, pearson r = 0.133

880
K562 true vs K562 pred, test data, pearson r = 0.695

880
K562 true vs A549 pred, test data, pearson r = 0.386

880
K562 true vs HCT116 pred, test data, pearson r = 0.447

880
A549 true vs HepG2 pred, test data, pearson r = 0.484

880
A549 true vs K562 pred, test data, pearson r = 0.390

880
A549 true vs A549 pred, test data, pearson r = 0.585

880
A549 true vs HCT116 pred, test data, pearson r = 0.524

880
HCT116 true vs HepG2 pred, test data, pearson r = 0.549

880
HCT116 true vs K562 pred, test data, pearson r = 0.445

880
HCT116 true vs A549 pred, test data, pearson r = 0.578

880
HCT116 true vs HCT116 pred, test data, pearson r = 0.654

43603
HepG2 true vs HepG2 pred, test data, pears